In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls drive/'My Drive'/'WEB_Ask_06devbros-main'/'ai'/'chatbot'

ls: cannot access 'drive/My Drive/WEB_Ask_06devbros-main/ai/chatbot': No such file or directory


In [ ]:
#!pip install -r drive/'My Drive'/'WEB_Ask_06devbros-main'/'ai'/'chatbot'/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'drive/My Drive/WEB_Ask_06devbros-main/ai/chatbot/requirements.txt'


In [ ]:
#!pip install --upgrade tokenizers==0.8.1.rc1

ERROR: Operation cancelled by user


In [ ]:
!pip install kogpt2-transformers
!pip install kobert-transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers==4.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 85.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.1
    Uninstalling tokenizers-0.15.1:
      Successfully uninstalled tokenizers-0.15.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot')

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import random

import torch
from transformers import (
  ElectraConfig,
  ElectraTokenizer
)
from model.koelectra import koElectraForSequenceClassification,koelectra_input

In [ ]:
def load_wellness_answer():
  root_path = '/content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot'
  category_path = f"{root_path}/data/wellness_dialog_category.txt"
  answer_path = f"{root_path}/data/wellness_dialog_answer.txt"

  c_f = open(category_path,'r')
  a_f = open(answer_path,'r')

  category_lines = c_f.readlines()
  answer_lines = a_f.readlines()

  category = {}
  answer = {}
  for line_num, line_data in enumerate(category_lines):
    data = line_data.split('    ')
    category[data[1][:-1]]=data[0]

  for line_num, line_data in enumerate(answer_lines):
    data = line_data.split('    ')
    keys = answer.keys()
    if(data[0] in keys):
      answer[data[0]] += [data[1][:-1]]
    else:
      answer[data[0]] =[data[1][:-1]]

  return category, answer

In [14]:

root_path='/content/drive/MyDrive/final_prj/WEB_Ask_06devbros-main/ai/chatbot'
checkpoint_path =f"{root_path}/checkpoint"
save_ckpt_path = f"{checkpoint_path}/koelectra-wellness-text-classification-25.pth"
model_name_or_path = "monologg/koelectra-base-discriminator"

#답변과 카테고리 불러오기
category = []
idx = -1
with open(root_path+'/data/wellness_data_for_text_classification.txt', 'r') as f:
  while True:
    line = f.readline()
    if not line:
      break
    datas = line.strip().split("\t")
    if datas[1] != str(idx):
      category.append(datas[2])
      idx += 1

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

# Electra Tokenizer
tokenizer = ElectraTokenizer.from_pretrained(model_name_or_path)

electra_config = ElectraConfig.from_pretrained(model_name_or_path)
model = koElectraForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path,
                                                            config=electra_config,
                                                            num_labels=359)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()


while 1:
  sent = input('\nQuestion: ') # '요즘 기분이 우울한 느낌이에요'
  data = koelectra_input(tokenizer,sent, device,512)
  # print(data)

  output = model(**data)

  logit = output
  softmax_logit = nn.Softmax(logit).dim
  softmax_logit = softmax_logit[0].squeeze()

  max_index = torch.argmax(softmax_logit).item()
  max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

  # answer_list = answer[category[str(max_index)]] #
  # answer_len= len(answer_list)-1 #
  # answer_index = random.randint(0,answer_len) #
  print(f'Answer: {answer_list[answer_index]}, index: {max_index}, value: {max_index_value}') #
  print(f'index: {category[max_index]}, value: {max_index_value}')
  print('-'*50)

# print('argmin:',softmax_logit[torch.argmin(softmax_logit)])

Some weights of the model checkpoint at monologg/koelectra-base-discriminator were not used when initializing koElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing koElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing koElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of koElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized


Question: 우울해


NameError: name 'answer_list' is not defined